In [1]:
!module list


Currently Loaded Modules:
  1) anaconda3/5.1.0-gcc/8.3.1     4) cudnn/8.0.0.180-11.0-linux-x64-gcc/7.5.0
  2) anaconda3/2019.10-gcc/8.3.1   5) openjdk/1.8.0_222-b10-gcc/8.3.1
  3) cuda/11.0.3-gcc/7.5.0         6) hadoop/3.2.1-gcc/8.3.1

 



In [2]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.utils import tf_utils
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle


import unicodedata
import re
import os
import time
import shutil
import requests
import tarfile
import glob

import pandas as pd
import numpy as np
import string, os
tf.__version__

# gpus = tf.config.list_physical_devices('GPU')
# gpu = gpus[0]

# tf.config.experimental.set_memory_growth(gpu, True)

'2.4.0'

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [4]:
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

In [5]:
file_name = "Project_CodeNet_LangClass.tar.gz"
data_url = f"https://dax-cdn.cdn.appdomain.cloud/dax-project-codenet/1.0.0/{file_name}"

# Download tar archive to local disk
with open(file_name, "wb") as f:
    f.write(requests.get(data_url).content)
    
# Extract contents of archive to local disk
if os.path.exists("data"):
    shutil.rmtree("data")    
with tarfile.open(file_name) as tfile:
    tfile.extractall()

In [6]:
!ls data data/train

data:
test  train

data/train:
C  C#  C++  D  Haskell	Java  JavaScript  PHP  Python  Rust


In [7]:
langs = [
  "C",
  "C#",
  "C++",
  "D",
  "Haskell",
  "Java",
  "JavaScript",
  "PHP",
  "Python",
  "Rust"
]

In [8]:
import os
import argparse
import re
from tokenize import tokenize, untokenize, COMMENT, STRING, NEWLINE, ENCODING, ENDMARKER, NL, INDENT, NUMBER
from io import BytesIO
import json

lits = json.load(open("literals.json"))

def process_string(token, special_chars={" ": "U+0020", ",": "U+002C"}):
    str_quote_options = ["'''", '"""', "'", '"']
    start_quote = ""
    end_quote = ""
    qualifier_regex = r"^[a-z]+"
    qualifier_match = re.search(qualifier_regex, token)
    # string qualifiers like 'r' for regex, 'f' for formatted string, 'b' for bytes, 'u' for unicode, etc (or combination of them)
    qualifier = "" if not qualifier_match else qualifier_match[0]
    # token string without qualifiers
    token_string = re.sub(qualifier_regex, "", token)
    # string literal without quotes
    str_lit = token_string
    for q in str_quote_options:
        if token_string.startswith(q):
            start_quote = q
            str_lit = str_lit[len(q) :]
            if token_string.endswith(q):
                end_quote = q
                str_lit = str_lit[: -len(q)]
            break
    # if start_quote in str_quote_options[:2]:
    #     return ""
    for sc in special_chars:
        str_lit = str_lit.replace(sc, special_chars[sc])
    return (
        f"{qualifier}{start_quote}<STR_LIT:{str_lit}>{end_quote}"
        if str_lit in lits['str']
        else f"{qualifier}{start_quote}<STR_LIT>{end_quote}"
    )

def py_tokenize(file_type):
    file_paths = glob.glob(os.path.join(os.getcwd(),"data/"+file_type+"/Python","*.*"))
    wf = open(os.path.join(os.getcwd(), f"{file_type}.txt"), 'w')
    local_corpus = []
    for path in file_paths:
        try:
            code = open(path).read()
            token_gen = tokenize(BytesIO(bytes(code, "utf8")).readline)
            out_tokens = []
            prev_eol = False
            for toknum, tokval, _, _, _ in token_gen:
                tokval = " ".join(tokval.split())
                if toknum == STRING:
                    add_token = process_string(tokval)
                    out_tokens.append(add_token)
                    prev_eol = False
                elif toknum == NUMBER:
                    if tokval in lits['num']:
                        out_tokens.append(f"<NUM_LIT:{tokval}>")
                    else:
                        out_tokens.append(f"<NUM_LIT>")
                    prev_eol = False
                elif toknum in [NEWLINE, NL]:
                    if not prev_eol:
                        out_tokens.append("<EOL>")
                        prev_eol = True
                elif toknum in [COMMENT, INDENT, ENCODING, ENDMARKER] or len(tokval) == 0:
                    continue
                else:
                    out_tokens.append(tokval)
                    prev_eol = False
            if out_tokens[0] == "<EOL>":
                out_tokens = out_tokens[1:]
            if out_tokens[-1] == "<EOL>":
                out_tokens = out_tokens[:-1]
        except Exception:
            out_tokens = []
#         local_corpus.extend((" ".join(out_tokens)).split('<EOL>'))
#         out_tokens = ["<s>"] + out_tokens + ["</s>"]
        out = " ".join(out_tokens)
        local_corpus.append(out)
        wf.write(out+"\n")
    print(f"{file_type}: are done")
    wf.close()
    return local_corpus

In [9]:
# file_list = glob.glob(os.path.join(os.getcwd(),"data/train/Python","*.*"))
# corpus = []

# for file_path in file_list:
#     with open(file_path) as f_input:
#         corpus.extend([line for line in f_input])
        
# print(corpus[40:50])
# py_tokenize("test")
corpus = py_tokenize("train")
# corpus2 = pd.DataFrame(corpus,columns=['code'])

train: are done


In [10]:
# corpus2.head()

In [11]:
# corpus2['tokenized']=corpus2['code'].str.split()
 
# corpus2.head()

In [76]:
full_corpus = ''.join(corpus)
print(full_corpus[0:300])

import sys <EOL> def prod2d ( vec1 , vec2 ) : <EOL> return vec1 [ <NUM_LIT:0> ] * vec2 [ <NUM_LIT:1> ] - vec1 [ <NUM_LIT:1> ] * vec2 [ <NUM_LIT:0> ] <EOL> lineNumber = <NUM_LIT:0> <EOL> for line in sys . stdin . readlines ( ) : <EOL> lineNumber += <NUM_LIT:1> <EOL> List = map ( float , line . strip 


In [13]:
corpus[0]

'import sys <EOL> def prod2d ( vec1 , vec2 ) : <EOL> return vec1 [ <NUM_LIT:0> ] * vec2 [ <NUM_LIT:1> ] - vec1 [ <NUM_LIT:1> ] * vec2 [ <NUM_LIT:0> ] <EOL> lineNumber = <NUM_LIT:0> <EOL> for line in sys . stdin . readlines ( ) : <EOL> lineNumber += <NUM_LIT:1> <EOL> List = map ( float , line . strip ( ) . split ( ) ) <EOL> nodes = [ [ List [ <NUM_LIT:0> ] , List [ <NUM_LIT:1> ] ] , [ List [ <NUM_LIT:2> ] , List [ <NUM_LIT:3> ] ] , [ List [ <NUM_LIT:4> ] , List [ <NUM_LIT:5> ] ] ] <EOL> point = [ List [ <NUM_LIT:6> ] , List [ <NUM_LIT:7> ] ] <EOL> ABvec = [ nodes [ <NUM_LIT:1> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:0> ] [ <NUM_LIT:0> ] , nodes [ <NUM_LIT:1> ] [ <NUM_LIT:1> ] - nodes [ <NUM_LIT:0> ] [ <NUM_LIT:1> ] ] <EOL> BCvec = [ nodes [ <NUM_LIT:2> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:1> ] [ <NUM_LIT:0> ] , nodes [ <NUM_LIT:2> ] [ <NUM_LIT:1> ] - nodes [ <NUM_LIT:1> ] [ <NUM_LIT:1> ] ] <EOL> CAvec = [ nodes [ <NUM_LIT:0> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:2> ] [ <NUM_LIT:0> ] , nodes

In [14]:
corpus_new = []
for code in corpus:
    corpus_new.extend(code.split('<EOL>'))
print(corpus_new[0:20])
print(len(corpus_new))

['import sys ', ' def prod2d ( vec1 , vec2 ) : ', ' return vec1 [ <NUM_LIT:0> ] * vec2 [ <NUM_LIT:1> ] - vec1 [ <NUM_LIT:1> ] * vec2 [ <NUM_LIT:0> ] ', ' lineNumber = <NUM_LIT:0> ', ' for line in sys . stdin . readlines ( ) : ', ' lineNumber += <NUM_LIT:1> ', ' List = map ( float , line . strip ( ) . split ( ) ) ', ' nodes = [ [ List [ <NUM_LIT:0> ] , List [ <NUM_LIT:1> ] ] , [ List [ <NUM_LIT:2> ] , List [ <NUM_LIT:3> ] ] , [ List [ <NUM_LIT:4> ] , List [ <NUM_LIT:5> ] ] ] ', ' point = [ List [ <NUM_LIT:6> ] , List [ <NUM_LIT:7> ] ] ', ' ABvec = [ nodes [ <NUM_LIT:1> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:0> ] [ <NUM_LIT:0> ] , nodes [ <NUM_LIT:1> ] [ <NUM_LIT:1> ] - nodes [ <NUM_LIT:0> ] [ <NUM_LIT:1> ] ] ', ' BCvec = [ nodes [ <NUM_LIT:2> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:1> ] [ <NUM_LIT:0> ] , nodes [ <NUM_LIT:2> ] [ <NUM_LIT:1> ] - nodes [ <NUM_LIT:1> ] [ <NUM_LIT:1> ] ] ', ' CAvec = [ nodes [ <NUM_LIT:0> ] [ <NUM_LIT:0> ] - nodes [ <NUM_LIT:2> ] [ <NUM_LIT:0> ] , nodes [ <NUM_LI

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 200
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=num_words, filters='')
tokenizer.fit_on_texts(corpus_new)

# Get our training data word index
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(corpus_new)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

# Output the results of our work
# print("Word index:\n", word_index)
print("\nTraining sequences:\n", train_sequences[0])
print("\nPadded training sequences:\n", train_padded[0])
# print("\nPadded training shape:", train_padded.shape)
# print("Training sequences data type:", type(train_sequences))
# print("Padded Training sequences data type:", type(train_padded))

pickle.dump(tokenizer, open('tokenizer-mix.pkl', 'wb'))


Training sequences:
 [45, 46]

Padded training sequences:
 [45 46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [36]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

746


In [17]:
print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_docs)

OrderedDict([('import', 100), ('sys', 97), ('def', 300), ('prod2d', 4), ('(', 2531), ('vec1', 11), (',', 2080), ('vec2', 11), (')', 2531), (':', 1435), ('return', 290), ('[', 2125), ('<num_lit:0>', 827), (']', 2125), ('*', 435), ('<num_lit:1>', 1341), ('-', 800), ('linenumber', 5), ('=', 1346), ('for', 415), ('line', 76), ('in', 422), ('.', 1012), ('stdin', 47), ('readlines', 10), ('+=', 355), ('list', 73), ('map', 53), ('float', 55), ('strip', 19), ('split', 85), ('nodes', 19), ('<num_lit:2>', 594), ('<num_lit:3>', 113), ('<num_lit:4>', 43), ('<num_lit:5>', 15), ('point', 55), ('<num_lit:6>', 15), ('<num_lit:7>', 19), ('abvec', 2), ('bcvec', 2), ('cavec', 2), ('apvec', 2), ('bpvec', 2), ('cpvec', 2), ('a', 546), ('b', 423), ('c', 295), ('if', 487), ('>', 92), ('and', 111), ('print', 185), ('"<str_lit>"', 74), ('elif', 57), ('<', 120), ('else', 108), ('gcd', 31), ('%', 49), ('lcm', 19), ('/', 161), ('data', 31), ('int', 124), ('d', 92), ('e', 70), ('f', 58), ('==', 249), ('y', 794), ('

In [73]:
test_corpus = []
for code in py_tokenize("test"):
    test_corpus.extend(code.split('<EOL>'))
print(test_corpus[0:20])
print(len(test_corpus))

test: are done
['import math , string , itertools , fractions , heapq , collections , re , array , bisect , sys , random , time , copy ', ' sys . setrecursionlimit ( <NUM_LIT:10> ** <NUM_LIT:7> ) ', ' inf = <NUM_LIT:10> ** <NUM_LIT:20> ', ' mod = <NUM_LIT:10> ** <NUM_LIT:9> + <NUM_LIT:7> ', ' def LI ( ) : return [ int ( x ) for x in sys . stdin . readline ( ) . split ( ) ] ', ' def LF ( ) : return [ float ( x ) for x in sys . stdin . readline ( ) . split ( ) ] ', ' def LS ( ) : return sys . stdin . readline ( ) . split ( ) ', ' def I ( ) : return int ( sys . stdin . readline ( ) ) ', ' def F ( ) : return float ( sys . stdin . readline ( ) ) ', ' def S ( ) : return input ( ) ', ' class Matrix ( ) : ', ' def __init__ ( self , A ) : ', ' self . A = A ', ' self . row = len ( A ) ', ' self . col = len ( A [ <NUM_LIT:0> ] ) ', ' def __iter__ ( self ) : ', ' return self . A . __iter__ ( ) ', ' def __getitem__ ( self , i ) : ', ' return self . A . __getitem__ ( i ) ', ' def __add__ ( self , B 

In [104]:
tokenizer2 = Tokenizer(num_words=num_words, filters='')
tokenizer2.fit_on_texts([full_corpus])

sequence_data = tokenizer2.texts_to_sequences([full_corpus])[0]
print(sequence_data[:50])

pickle.dump(tokenizer2, open('tokenizer-mix1.pkl', 'wb'))

[80, 46, 1, 28, 2, 6, 3, 7, 1, 29, 4, 12, 5, 21, 4, 9, 5, 13, 4, 9, 5, 21, 4, 12, 5, 1, 8, 12, 1, 24, 61, 23, 46, 10, 90, 10, 2, 3, 7, 1, 25, 9, 1, 63, 8, 81, 2, 77, 6, 61]


In [92]:
vocab_size = len(tokenizer2.word_index) + 1
print(vocab_size)

774


In [94]:
# print(tokenizer2.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_docs)

In [95]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  38387


array([[80, 46],
       [46,  1],
       [ 1, 28],
       [28,  2],
       [ 2,  6],
       [ 6,  3],
       [ 3,  7],
       [ 7,  1],
       [ 1, 29],
       [29,  4]])

In [96]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

In [97]:
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

The Data is:  [80 46  1 28  2]
The responses are:  [46  1 28  2  6]


In [98]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [99]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [100]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1, 10)             7740      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_5 (Dense)              (None, 774)               774774    
Total params: 13,831,514
Trainable params: 13,831,514
Non-trainable params: 0
_________________________________________________________________


In [101]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword-mix1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [102]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

In [103]:
model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/150
600/600 [==============================] - 13s 17ms/step - loss: 4.0644

Epoch 00001: loss improved from inf to 3.47923, saving model to nextword-mix1.h5
Epoch 2/150
600/600 [==============================] - 10s 17ms/step - loss: 2.8405

Epoch 00002: loss improved from 3.47923 to 2.79597, saving model to nextword-mix1.h5
Epoch 3/150
600/600 [==============================] - 10s 17ms/step - loss: 2.6715

Epoch 00003: loss improved from 2.79597 to 2.66644, saving model to nextword-mix1.h5
Epoch 4/150
600/600 [==============================] - 10s 17ms/step - loss: 2.5968

Epoch 00004: loss improved from 2.66644 to 2.60510, saving model to nextword-mix1.h5
Epoch 5/150
600/600 [==============================] - 10s 17ms/step - loss: 2.5662

Epoch 00005: loss improved from 2.60510 to 2.57011, saving model to nextword-mix1.h5
Epoch 6/150
600/600 [==============================] - 10s 17ms/step - loss: 2.5408

Epoch 00006: loss improved from 2.57011 to 2.53698, saving model to n

In [105]:
from tensorflow.keras.models import load_model
model = load_model('nextword-mix1.h5')
tokenizer = pickle.load(open('tokenizer-mix1.pkl', 'rb'))

In [112]:
def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
        print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [118]:
Predict_Next_Words(model,tokenizer,"import")

[56]
math


'math'